# Analysing Dataset

In [1]:
import pandas as pd
import pandas_profiling
import requests
#from functional import pseq
import pathlib
import os
import json
import time
import datetime

In [2]:
directory_liar_dataset = "../liar_dataset"
directory_statements = f"{directory_liar_dataset}/statements"
directory_visualizations = "../visualizations"

In [30]:
df = pd.concat([pd.read_csv(f"{directory_liar_dataset}/{part}.tsv", sep='\t', header=None) for part in ['train', 'valid']])
df.columns = ['statement_id', 'label', 'statement', 'subject', 'speaker', 'speakers_job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context']

df.statement_id = df.statement_id.apply(lambda x: x[:-5])  # remove .json and get just ID

In [4]:
df.head(10)

,statement_id,label,statement,subject,speaker,speakers_job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context
0,2635,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
5,12465,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
6,2342,barely-true,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release.
7,153,half-true,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa."
8,5602,half-true,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website
9,9741,mostly-true,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video


In [74]:
#new Import data file
county_raw = pd.read_csv("../data/acs2015_county_data.csv")
US_states = county_raw['State'].unique()
US_states

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'Puerto Rico'],
      dtype=object)

In [ ]:
for i in range(1, county_raw.shape()):
    b=current_tracks.rel_values.iloc[i]

In [95]:
county_raw.shape[0]

(3220, 37)

In [75]:
dfjoin = US_counties.join(df, how='left', on='')

ValueError: can not merge DataFrame with instance of type <class 'str'>

In [64]:
#new Biggest party_affiliation liars pants on fire
df['sum_not_so_true'] = df['pants_on_fire_counts']/(df['barely_true_counts'] + df['false_counts'] + df['half_true_counts'] + df['mostly_true_counts'] + df['pants_on_fire_counts'])
number_of_party_affiliation = df.groupby('party_affiliation')['sum_not_so_true'].sum().sort_values(ascending=False)
number_of_party_affiliation

party_affiliation
republican                      478.423916
none                            325.463424
democrat                        198.573266
organization                     18.833333
newsmaker                         5.250000
talk-show-host                    4.500000
independent                       3.666667
libertarian                       3.333333
business-leader                   2.000000
journalist                        1.666667
green                             1.500000
columnist                         1.181818
tea-party-member                  1.000000
labor-leader                      1.000000
constitution-party                0.666667
government-body                   0.000000
education-official                0.000000
democratic-farmer-labor           0.000000
liberal-party-canada              0.000000
county-commissioner               0.000000
ocean-state-tea-party-action      0.000000
state-official                    0.000000
activist                          0.

#new Biggest party_affiliation liars pants on fire

Here are the party_affiliations who most lie ordered by their proportion of lies. But we already know that the 2 dominant parties in USA are republican and democrat. We see that there are lots of unknown party affiliations from which we can make identify 2 possibilities

In [51]:
#new Biggest people liars pants on fire
number_of_party_affiliation = df.groupby(['speaker'])['sum_not_so_true'].sum().sort_values(ascending=False)
number_of_party_affiliation.head(10)


speaker
chain-email         100.726744
donald-trump         58.006135
blog-posting         40.051282
facebook-posts       33.702128
mitt-romney          21.051136
rick-perry           19.636364
michele-bachmann     15.428571
scott-walker         12.173333
viral-image          11.785714
newt-gingrich        11.763889
Name: sum_not_so_true, dtype: float64

#new Biggest people liars

With no big astonishment, we can elect Trump as the biggest liar. He is far ahead from the others...

In [93]:
#new clean field "context"
#US_states = county_raw['State'].unique()
all_contexts = df['context'].unique()
all_contexts.shape
nb_elements_context = df.groupby(['context'])['context'].count().sort_values(ascending=False)
nb_elements_context.head(50)

context
a news release                            276
an interview                              254
a press release                           251
a speech                                  236
a TV ad                                   200
a tweet                                   171
a campaign ad                             151
a television ad                           138
a radio interview                         118
a debate                                  102
a news conference                          91
a press conference                         81
a campaign commercial                      81
a Facebook post                            80
a television interview                     74
a speech.                                  68
a press release.                           53
a radio ad                                 53
a TV ad.                                   53
a TV interview                             50
a chain e-mail                             49
an interview.             

In [53]:
#new Most favorable context for liars pants on fire
liar_context = df.groupby(['context'])['sum_not_so_true'].sum().sort_values(ascending=False)
liar_context.head(20)


context
a chain e-mail            28.691860
a news release            25.419554
a tweet                   20.383328
an interview              19.350394
a chain email             18.267591
a press release           17.039260
a TV ad                   15.893148
a Facebook post           15.513838
a speech                  14.586103
a television ad           12.484779
a campaign ad             11.740079
a radio interview          9.007631
a debate                   8.055450
a meme on social media     7.791053
a campaign commercial      6.712491
an email                   6.675616
a TV ad.                   6.644953
a blog post                5.543142
a campaign mailer          5.508932
a speech.                  5.350963
Name: sum_not_so_true, dtype: float64

#new Most favorable context for liars pants on fire

Mails is the most favorable context. Tweets and facebook appear only later in the list.

In [6]:
# form URL from statement ID
def get_URL(statement_id):
    return f"http://www.politifact.com/api/v/2/statement/{statement_id}/?format=json"

In [7]:
def extract_information(res):
    try:
        author = res['author']

        try:
            if len(author) > 0:
                author = author[0]['name_slug']
            else:
                author = None
        except Exception:
            print(author)

        return {'author_name_slug': author,
                'ruling_date':  res['ruling_date'],
                'statement_date' :res['statement_date'],
                'speaker_current_job': res['speaker']['current_job'],
                'speaker_first_name': res['speaker']['first_name'],
                'speaker_last_name': res['speaker']['last_name'],
                'speaker_home_state': res['speaker']['home_state'],
                'statement_id': res['id']
               }
    except KeyError:
        return {}

In [8]:
#with requests.Session() as session:
#    additional_information = statement_ids.map(lambda sid: session.get(get_URL(sid)))\
#                                          .filter(lambda r: r.ok)\
#                                          .map(lambda r: r.json())\
#                                          .map(extract_information)\
#                                          .to_pandas()

In [9]:
def safe_json_read(f):
    try:
        with open(f, 'r') as fc:
            return json.load(fc)
    except json.JSONDecodeError:
        print(f)
        return {}

In [10]:
additional_information = pseq(pathlib.Path('../liar_dataset/statements/').iterdir())\
                               .map(safe_json_read)\
                               .filter(lambda x: len(x) > 0)\
                               .map(extract_information)\
                               .to_pandas()

additional_information['statement_date'] = pd.to_datetime(additional_information['statement_date'])

NameError: name 'pseq' is not defined

In [ ]:
def label_to_nb(l): 
    return ['true', 'mostly-true', 'half-true', 'barely-true', 'false', 'pants-fire'].index(l)

df['label_as_nb'] = df['label'].apply(label_to_nb) * 2 # TODO think about this, this will give the false-hoods more weight

In [ ]:
df['statement_id'] = pd.to_numeric(df['statement_id'])
lies = df.merge(additional_information, on='statement_id', how='left')

In [ ]:
lies.loc[lies['speaker'] == 'barack-obama', ]['pants_on_fire_counts'].value_counts()

In [ ]:
lies[lies['speakers_job_title'].str.contains('County') == True].shape

In [ ]:
lies['statement_date'].describe()

In [ ]:
pandas_profiling.ProfileReport(lies)

# federal election results

In [ ]:
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 300

In [ ]:
from itertools import product
from functools import reduce

In [ ]:
def add_ending(f):
    if '2016' in f:
        return f"{f}x"
    else:
        return f
    
# TODO do 2012 it's a special snowflake
election_files = [(add_ending(f'../data/election_results/federalelections{year}.xls'), year) for year in [2014, 2016]]

In [ ]:
election_results_cols_of_interest = ['CANDIDATE NAME', 'PRIMARY VOTES', 'PRIMARY %']

def fix_columns_election_results(df, year, type_):
    df = df.loc[:, election_results_cols_of_interest]
    df[f'primary_votes_{type_.lower()}_{year}'] = df['PRIMARY VOTES']
    df[f'primary_votes_{type_.lower()}_{year}_pct'] = df['PRIMARY %']
    return df.drop(columns=['PRIMARY VOTES', 'PRIMARY %'])


def get_only_voting_results(df):
    return df.loc[df['CANDIDATE NAME'].notna() & df['PRIMARY VOTES'].notna() & df['CANDIDATE NAME'].ne('Scattered') & df['CANDIDATE NAME'].ne('All Others'), :]


def prep_election_results(df, year, type_):
    return fix_columns_election_results(get_only_voting_results(df), year, type_)

In [ ]:
election_results = [prep_election_results(pd.read_excel(f, sheet_name=f'{year} US {type_} Results by State'), year, type_) for (f, year), type_ in product(election_files, ['Senate', 'House'])]

election_results = reduce(lambda acc, el: pd.merge(acc, el, on='CANDIDATE NAME', how='outer'), election_results)

In [ ]:
# yeah ... let's see how many we can join. the one letter endings might be a problem
election_results['CANDIDATE NAME'].value_counts()

In [ ]:
# we are only interest in people and they have a first name
lies = lies.loc[lies['speaker_first_name'].notnull(), :]

In [ ]:
# to aggregate the statements
lies['statement_year'] = lies['statement_date'].dt.year

# for the merging
lies['speaker_full_name'] = lies['speaker_last_name'] + ', ' + lies['speaker_first_name']

In [ ]:
# todo expand this and check this! this is just a quick and dirty fix
# is it really houseman? probably not...
_job_titles_of_interest = [('senat', 'senator'), ('governor', None), ('congress', 'congressman'), ('mayor', None), ('president', None), ('house', 'houseman'), ('rep', 'houseman')]
job_titles_of_interest = [out if out is not None else j for j, out in _job_titles_of_interest]

def cleaned_job_title(jt):
    jt = str(jt).lower()
    
    for j, out in _job_titles_of_interest:
        if j in jt:
            return out if out is not None else j
    else:
        return jt

lies['speakers_job_title_cleaned'] = lies['speakers_job_title'].apply(cleaned_job_title)

In [ ]:
_t = lies.merge(election_results, left_on='speaker_full_name', right_on='CANDIDATE NAME', how='outer')

In [ ]:
print(f"found election results for {_t['CANDIDATE NAME'].notnull().sum()} ({_t['CANDIDATE NAME'].notnull().mean()}%) people")

In [ ]:
votes_cols = [c for c in _t.columns if 'votes' in c]
useful_idx = reduce(lambda acc, el: acc | el, [_t[c].notnull() for c in votes_cols]) & _t['speaker'].notnull()

print(f"found useful results for {useful_idx.sum()} people")

columns_of_interest = ['label', 'label_as_nb', 'subject', 'speaker', 'speakers_job_title_cleaned', 'state_info', 'party_affiliation', 'context', 'statement_date'] + votes_cols
_t.loc[useful_idx, columns_of_interest]

In [ ]:
_t.loc[useful_idx, 'speakers_job_title_cleaned'].value_counts()

In [ ]:
_t.loc[_t['speakers_job_title_cleaned'].isin(job_titles_of_interest) & useful_idx, columns_of_interest]

# DATA SET COMPLETE

In [ ]:
median_speaker_value = _t.groupby(['statement_year', 'speaker'])['label_as_nb'].median().reset_index()

In [ ]:
median_speaker_value[median_speaker_value['statement_year'] == 2016]

## One row analysis

Let's analyse first row, statement with id `1`. What is the information we get there?

In [ ]:
sid = '1'

In [ ]:
df[df.statement_id == sid]

In [ ]:
with open(f"{directory_statements}/{sid}.json", "r") as f:
    data = json.load(f)
data

Just to visualize JSON hierarchy, run the following cell:

In [ ]:
def go_further(dic, name):
    dict_vis = {"name": name, "children": []}
    for k, v in dic.items():
        if type(v) == str:
            new_el = {"name": k}
        elif type(v) == list:
            if len(v) > 0:
                new_el = go_further(v[0], k)
        elif type(v) == dict:
            new_el = go_further(v, k)
        else:
            new_el = {"name": k}
        dict_vis["children"].append(new_el)
        
    return dict_vis

my_dict = go_further(data, name="statement_info")

with open(f"{directory_visualizations}/data.json", "w") as f:
    json.dump(my_dict, f)

print(f"Checkout visualization by: \n1) cd ../visualizations \n2) python -m http.server \n3) in browser, open: http://localhost:8000/")